Note: I left the code to generate the htfs for each test system in the notebook, but if you don't want to re-generate, you can use the pickled htf that I saved (I left the full path in for you). I generated these htfs based after you added support for `lambda_{i}_bonds_old/new` and `scale_lambda_0_bonds`

In [1]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle
import copy

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


INFO:rdkit:Enabling RDKit 2021.03.4 jupyter extensions


conducting subsequent work with the following platform: CUDA


In [2]:
def test_angle_energies(htf, is_old=True, is_solvated=False, check_scale=False):
    htf_copy = copy.deepcopy(htf)
    
    # Get harmonic angle force and old/new positions
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    harmonic_angle_force = system.getForce(1) 
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)
    
    # Get custom angle force and hybrid positions
    angle_force_index = 2 if is_solvated else 1
    hybrid_system = htf.hybrid_system
    custom_angle_force = hybrid_system.getForce(angle_force_index)
    hybrid_positions = htf.hybrid_positions
    
    # Remove nonbonded/exception forces
    for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 3, -1):
        hybrid_system.removeForce(i - 1)
        
    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    for i in range(custom_angle_force.getNumGlobalParameters()):
        if custom_angle_force.getGlobalParameterName(i) == 'lambda_alchemical_angles_old':
            custom_angle_force.setGlobalParameterDefaultValue(i, lambda_old)
        if custom_angle_force.getGlobalParameterName(i) == 'lambda_alchemical_angles_new':
            custom_angle_force.setGlobalParameterDefaultValue(i, lambda_new)

    # Zero the unique old/new angles in the custom bond force
    hybrid_to_angle_indices = htf._hybrid_to_new_angle_indices if is_old else htf._hybrid_to_old_angle_indices
    for hybrid_idx, idx in hybrid_to_angle_indices.items():
        p1, p2, p3, hybrid_params = custom_angle_force.getAngleParameters(hybrid_idx)
        hybrid_params = list(hybrid_params)
        hybrid_params[-1] *= 0
        hybrid_params[-2] *= 0
        custom_angle_force.setAngleParameters(hybrid_idx, p1, p2, p3, hybrid_params)

    ## Get energy components of standard bond force
    platform = configure_platform(REFERENCE_PLATFORM)
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta)
    print(components_other)
    
    ## Get energy components of custom bond force
    if is_solvated:
        custom_angle_force.setUsesPeriodicBoundaryConditions(True)
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta)
    print(components_hybrid)
    
    assert np.isclose([components_other[1][1]], [components_hybrid[1][1]])
    
    print("Success! Custom angle force and standard angle force energies are equal!")
    
    if check_scale:
        
        # Get custom bond force and hybrid positions
        angle_force_index = 2 if is_solvated else 1
        hybrid_system = htf_copy.hybrid_system
        custom_angle_force = hybrid_system.getForce(angle_force_index)
        hybrid_positions = htf_copy.hybrid_positions
        
        if is_solvated:
            custom_angle_force.setUsesPeriodicBoundaryConditions(True)
        
        # Remove nonbonded/exception forces
        for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 3, -1):
            hybrid_system.removeForce(i - 1)
        
         # Zero the unique old/new angles in the custom bond force
        hybrid_to_angle_indices = htf_copy._hybrid_to_new_angle_indices if is_old else htf_copy._hybrid_to_old_angle_indices
        for hybrid_idx, idx in hybrid_to_angle_indices.items():
            p1, p2, p3, hybrid_params = custom_angle_force.getAngleParameters(hybrid_idx)
            hybrid_params = list(hybrid_params)
            hybrid_params[-1] *= 0
            hybrid_params[-2] *= 0
            custom_angle_force.setAngleParameters(hybrid_idx, p1, p2, p3, hybrid_params)

        ## Get energy components of custom bond force
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid)
        
        # Set `scale_lambda_{i}` to 0.5
        for i in range(custom_angle_force.getNumGlobalParameters()):
            if custom_angle_force.getGlobalParameterName(i) == 'lambda_rest_angles':
                custom_angle_force.setGlobalParameterDefaultValue(i, 0.5)
        
        ## Get energy components of custom bond force iwth scaling
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid_scaled = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid_scaled)
        
        assert not np.isclose([components_hybrid[1][1]], [components_hybrid_scaled[1][1]])
        
        print("Success! Scaling the angle force changes the energy")

# Alanine dipeptide in vacuum

### Test one alchemical region, no rest regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [2]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [4]:
test_angle_energies(htf)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034551762541440684), ('HarmonicAngleForce', 0.6072023625714381), ('PeriodicTorsionForce', 16.176828968049858), ('NonbondedForce', -39.18692822729614), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.10254177146136774), ('CustomAngleForce', 0.60720236257148), ('CustomTorsionForce', 46.653838127510966), ('CustomNonbondedForce', -134.40418217693514), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [5]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [6]:
test_angle_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.10254177146136774), ('HarmonicAngleForce', 6.614990660768418), ('PeriodicTorsionForce', 25.94279472193764), ('NonbondedForce', 479.64425488269234), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.10254177146136774), ('CustomAngleForce', 6.614990660768418), ('CustomTorsionForce', 46.653838127510966), ('CustomNonbondedForce', -134.40418217693514), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one rest region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [7]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [8]:
htf.hybrid_system.getForces()

[<openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ad0b7980330> >,
 <openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2ad0b79803f0> >,
 <openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2ad0b79801b0> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ad0b7980270> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ad0b7980060> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ad0b79800f0> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2ad0b7980030> >]

In [9]:
test_angle_energies(htf, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0345517625414457), ('HarmonicAngleForce', 0.6072023625714801), ('PeriodicTorsionForce', 16.176828968049836), ('NonbondedForce', -39.186928227296285), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.7760347658341544), ('CustomAngleForce', 0.60720236257148), ('CustomTorsionForce', 46.45962220603571), ('CustomNonbondedForce', -134.40418217693514), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.7760347658341544), ('CustomAngleForce', 0.60720236257148), ('CustomTorsionForce', 46.45962220603571), ('CustomNonbondedForce', -134.40418217693514), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForc

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [10]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [11]:
test_angle_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.6716646808138056), ('HarmonicAngleForce', 271.4974073413748), ('PeriodicTorsionForce', 28.667471150475443), ('NonbondedForce', 42.65813680526489), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.6716646808138056), ('CustomAngleForce', 271.4974073413748), ('CustomTorsionForce', 21.70598622028408), ('CustomNonbondedForce', -134.40418217693514), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


# Alanine dipeptide in solvent

### Test one alchemical region, no rest regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [10]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [11]:
htf.hybrid_system.getForces()

[<openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2ad0b787b780> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ad0b787b240> >,
 <openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2ad0b787b8d0> >,
 <openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2ad0b787b5d0> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ad0b787b690> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ad0b787bc00> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ad0b787b3f0> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2ad0b787bba0> >]

In [12]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03455176254144657), ('HarmonicAngleForce', 0.6072023625714819), ('PeriodicTorsionForce', 16.176828968049854), ('NonbondedForce', -952.8311977716749), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.24233683996148545), ('CustomAngleForce', 0.6072023625714819), ('CustomTorsionForce', 46.50113378965608), ('CustomNonbondedForce', -7438.971401447827), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [13]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [14]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.24233683996148545), ('HarmonicAngleForce', 5.819583388911687), ('PeriodicTorsionForce', 26.744150017390197), ('NonbondedForce', 3310.8084258628214), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.24233683996148545), ('CustomAngleForce', 5.819583388911687), ('CustomTorsionForce', 46.50113378965608), ('CustomNonbondedForce', -7438.971401447827), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [15]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [16]:
test_angle_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03455176254144657), ('HarmonicAngleForce', 0.6072023625714819), ('PeriodicTorsionForce', 16.176828968049854), ('NonbondedForce', -913.2000212841806), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.14844046258249663), ('CustomAngleForce', 0.6072023625714819), ('CustomTorsionForce', 45.71638366089612), ('CustomNonbondedForce', -7431.170228193835), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.14844046258249663), ('CustomAngleForce', 0.6072023625714819), ('CustomTorsionForce', 45.71638366089612), ('CustomNonbondedForce', -7431.170228193834), ('AndersenThermostat', 0.0), ('MonteCarloBaro

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [17]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [18]:
test_angle_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.14844046258249663), ('HarmonicAngleForce', 16.191148224923904), ('PeriodicTorsionForce', 26.927581144364517), ('NonbondedForce', 354932.57371438603), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.14844046258249663), ('CustomAngleForce', 16.191148224923907), ('CustomTorsionForce', 45.71638366089612), ('CustomNonbondedForce', -7431.170228193835), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.14844046258249663), ('CustomAngleForce', 16.191148224923907), ('CustomTorsionForce', 45.71638366089612), ('CustomNonbondedForce', -7431.170228193835), ('AndersenThermostat', 0.0), ('MonteCarloBaro

# CDK2 transformation in solvent

### Test one alchemical region, no rest regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [20]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [21]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.5773624247683875), ('HarmonicAngleForce', 29.004860118481027), ('PeriodicTorsionForce', 14.976236339333706), ('NonbondedForce', 1823.5369961225852), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 29.004860118481027), ('CustomTorsionForce', 14.976242103332332), ('CustomNonbondedForce', -11649.155520306096), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [22]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [23]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.489378129340331), ('HarmonicAngleForce', 30.340922034722173), ('PeriodicTorsionForce', 14.999853713142482), ('NonbondedForce', 1833.8868610149345), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 30.34092203472218), ('CustomTorsionForce', 14.976242103332332), ('CustomNonbondedForce', -11649.155520306096), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one rest region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [24]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [25]:
test_angle_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.5773624247683875), ('HarmonicAngleForce', 29.004860118481027), ('PeriodicTorsionForce', 14.976236339333706), ('NonbondedForce', 1056.4211043395715), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 29.004860118481027), ('CustomTorsionForce', 14.976242103332332), ('CustomNonbondedForce', -11613.058217647602), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 29.004860118481027), ('CustomTorsionForce', 14.976242103332332), ('CustomNonbondedForce', -11613.058217647602), ('AndersenThermostat', 0.0), ('MonteCarloBar

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [26]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [27]:
test_angle_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.489378129340331), ('HarmonicAngleForce', 29.107274175149552), ('PeriodicTorsionForce', 15.003700186989857), ('NonbondedForce', 1067.1653196416876), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 29.10727417514956), ('CustomTorsionForce', 14.976242103332332), ('CustomNonbondedForce', -11613.0582176476), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 29.10727417514956), ('CustomTorsionForce', 14.976242103332332), ('CustomNonbondedForce', -11613.0582176476), ('AndersenThermostat', 0.0), ('MonteCarloBarostat',

# 8mer in solvent

### Test one alchemical region, no rest regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [24]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [30]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153172), ('HarmonicAngleForce', 12.445685055931843), ('PeriodicTorsionForce', 156.2435090637808), ('NonbondedForce', -9963.845239675724), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153172), ('CustomAngleForce', 12.445685055931843), ('CustomTorsionForce', 156.24352300681832), ('CustomNonbondedForce', -27470.062373416964), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [31]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)


In [32]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 9.361655470384113), ('HarmonicAngleForce', 14.18827616458745), ('PeriodicTorsionForce', 174.97365990635137), ('NonbondedForce', -9771.178805994987), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153172), ('CustomAngleForce', 14.188276164587446), ('CustomTorsionForce', 156.24352300681832), ('CustomNonbondedForce', -27470.062373416964), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [45]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [46]:
test_angle_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153172), ('HarmonicAngleForce', 12.445685055931843), ('PeriodicTorsionForce', 156.2435090637808), ('NonbondedForce', -10206.31135544179), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153172), ('CustomAngleForce', 12.445685055931843), ('CustomTorsionForce', 156.24352300681832), ('CustomNonbondedForce', -27503.63689266388), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153172), ('CustomAngleForce', 12.445685055931843), ('CustomTorsionForce', 156.24352300681832), ('CustomNonbondedForce', -27503.63689266388), ('AndersenThermostat', 0.0), ('MonteCarloBarostat'

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [47]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [48]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 8.6173683680896), ('HarmonicAngleForce', 27.500492144184243), ('PeriodicTorsionForce', 163.92624538706607), ('NonbondedForce', 1477052.424363085), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153172), ('CustomAngleForce', 27.500492144184243), ('CustomTorsionForce', 156.24352300681832), ('CustomNonbondedForce', -27503.63689266387), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


## Barnase:barstar (no naked charge fix)

### Test one alchemical region, no rest regions

#### Old system HarmonicAngleForce vs hybrid system CustomAngleForce

In [28]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [29]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 569.5926176486894), ('HarmonicAngleForce', 1323.9862513751366), ('PeriodicTorsionForce', 3795.5154898495653), ('NonbondedForce', 57388502.27274529), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.5926176486894), ('CustomAngleForce', 1323.9862513751366), ('CustomTorsionForce', 3806.242160413789), ('CustomNonbondedForce', -171553.91100476545), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicAngleForce vs hybrid system CustomAngleForce

In [30]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [31]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 569.2585424303023), ('HarmonicAngleForce', 1516.3484097339963), ('PeriodicTorsionForce', 3785.296157714871), ('NonbondedForce', 57388582.02854316), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.5926176486894), ('CustomAngleForce', 1516.3484097339963), ('CustomTorsionForce', 3806.242160413789), ('CustomNonbondedForce', -171553.91100476545), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one rest region

#### Old system HarmonicAngleForce vs hybrid system CustomAngleForce

In [32]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [33]:
test_angle_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 569.5926176486894), ('HarmonicAngleForce', 1323.9862513751366), ('PeriodicTorsionForce', 3795.5154898495653), ('NonbondedForce', 57389039.21388332), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.5926176486894), ('CustomAngleForce', 1323.9862513751366), ('CustomTorsionForce', 3806.2139744949422), ('CustomNonbondedForce', -171254.5397431931), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.5926176486894), ('CustomAngleForce', 1323.9862513751366), ('CustomTorsionForce', 3806.2139744949422), ('CustomNonbondedForce', -171254.5397431931), ('AndersenThermostat', 0.0), ('MonteCarloBarostat'

#### New system HarmonicAngleForce vs hybrid system CustomAngleForce

In [34]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [35]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 569.2585424303023), ('HarmonicAngleForce', 1527.3093929656204), ('PeriodicTorsionForce', 3785.289987895967), ('NonbondedForce', 57389118.86896432), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.5926176486894), ('CustomAngleForce', 1527.30939296562), ('CustomTorsionForce', 3806.2139744949422), ('CustomNonbondedForce', -171254.5397431931), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


In [ ]:
# TO DO: actually test the scaling of the energies matches